In [1]:
import ccxt
from datetime import datetime, timedelta, timezone
import sys
import pandas as pd
import time
import numpy as np
import ta
import pytz

In [2]:

def check_1_min_apart(a):

    a['time_diff'] = a['timestamp'].diff()

    # Check if all differences are exactly one minute
    one_minute = pd.Timedelta(minutes=15)
    is_one_minute_apart = a['time_diff'].dropna().eq(one_minute).all()  # dropna to ignore the first NaT result

    print("All timestamps are exactly one minute apart:", is_one_minute_apart)
    

In [3]:
exchange = ccxt.okx({
    'enableRateLimit': True  # It's important to handle rate limiting
})

In [4]:
exchange.apiKey = "40fb3b5f-d8b2-45c4-891c-9b185223c0d5"
exchange.secret = "F058FB6F168A9260FE2F41B4F1AC60C6"
exchange.password = "!Qaz1354403003"

In [5]:
def get_data(since, end_date): 

    symbol = 'SOL-USDT-SWAP'
    timeframe = '15m'
    all_data = pd.DataFrame()
    
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe)
    
    while since < end_date:
        try:
            print(f"Fetching data since {exchange.iso8601(since)}")
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
            if ohlcv:
                # Convert to DataFrame and append
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                all_data = pd.concat([all_data, df], ignore_index=True)

                # Update 'since' to just after the last fetched timestamp to continue from there
                since = ohlcv[-1][0] + exchange.parse_timeframe(timeframe) * 1000  # add one minute in ms
            else:
                # If no data returned, advance time by a reasonable chunk (here 1 day in milliseconds)
                since += 24 * 60 * 60 * 1000
        except ccxt.NetworkError as e:
            print(f"Network error: {e}")
            continue
        except ccxt.ExchangeError as e:
            print(f"Exchange error: {e}")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break
    return all_data

In [12]:
since = exchange.parse8601('2024-01-01T00:00:00Z') 
end_date = exchange.parse8601('2024-06-28T00:00:00Z')

In [13]:
new_data= get_data(since, end_date)

Fetching data since 2024-01-01T00:00:00.000Z
Fetching data since 2024-01-02T01:00:00.000Z
Fetching data since 2024-01-03T02:00:00.000Z
Fetching data since 2024-01-04T03:00:00.000Z
Fetching data since 2024-01-05T04:00:00.000Z
Fetching data since 2024-01-06T05:00:00.000Z
Fetching data since 2024-01-07T06:00:00.000Z
Fetching data since 2024-01-08T07:00:00.000Z
Fetching data since 2024-01-09T08:00:00.000Z
Fetching data since 2024-01-10T09:00:00.000Z
Fetching data since 2024-01-11T10:00:00.000Z
Fetching data since 2024-01-12T11:00:00.000Z
Fetching data since 2024-01-13T12:00:00.000Z
Fetching data since 2024-01-14T13:00:00.000Z
Fetching data since 2024-01-15T14:00:00.000Z
Fetching data since 2024-01-16T15:00:00.000Z
Fetching data since 2024-01-17T16:00:00.000Z
Fetching data since 2024-01-18T17:00:00.000Z
Fetching data since 2024-01-19T18:00:00.000Z
Fetching data since 2024-01-20T19:00:00.000Z
Fetching data since 2024-01-21T20:00:00.000Z
Fetching data since 2024-01-22T21:00:00.000Z
Fetching d

In [16]:
new_data.tail()

,timestamp,open,high,low,close,volume,time_diff
17190,2024-06-28 01:30:00,149.34,150.92,149.27,149.67,173743.89,0 days 00:15:00
17191,2024-06-28 01:45:00,149.67,149.69,148.63,148.95,161093.73,0 days 00:15:00
17192,2024-06-28 02:00:00,148.96,149.36,148.00,148.08,87390.18,0 days 00:15:00
17193,2024-06-28 02:15:00,148.08,148.68,147.22,147.27,107710.84,0 days 00:15:00
17194,2024-06-28 02:30:00,147.27,147.56,146.82,147.49,126425.62,0 days 00:15:00


In [17]:
check_1_min_apart(new_data)

All timestamps are exactly one minute apart: True


In [18]:
new_data.to_csv('SOL_15_minute.csv', index=False)